## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
data_dir = "../../Data/Weather"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [4]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3
0,-0.002153,-0.182283,-0.031488,-0.057910
1,-0.004001,-0.165204,-0.052665,-0.042697
2,-0.000222,-0.218895,-0.067369,0.053234
3,-0.006350,-0.154621,-0.076194,0.102126
4,-0.018138,-0.181790,-0.110199,0.060141


In [5]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00488209', u'USC00488830', u'USC00488889', u'USC00480228', u'USS0009F27S']


,0,1,2,3
station,,,,
USC00488209,-0.002153,-0.182283,-0.031488,-0.057910
USC00488830,-0.004001,-0.165204,-0.052665,-0.042697
USC00488889,-0.000222,-0.218895,-0.067369,0.053234
USC00480228,-0.006350,-0.154621,-0.076194,0.102126
USS0009F27S,-0.018138,-0.181790,-0.110199,0.060141


In [6]:
table=Eig.join(Stations,how='left')

In [7]:
#table=table[['name','latitude','longitude','elevation',0,1,2,3,4,5,6,7]]
table=table[['name','latitude','longitude','elevation',0,1,2,3]]
mlat = round(table.latitude.min(),4)
mxlat = round(table.latitude.max(),4)
mlong = round(table.longitude.min(),4)
mxlong = round(table.longitude.max(),4)
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3
station,,,,,,,,
USC00488209,SHOSHONI,43.2372,-108.1097,1474.3,-0.002153,-0.182283,-0.031488,-0.057910
USC00488830,TECKLA 3 E,43.5833,-105.2833,1457.9,-0.004001,-0.165204,-0.052665,-0.042697
USC00488889,THERMOPOLIS 27 WNW,43.7167,-108.7500,1795.9,-0.000222,-0.218895,-0.067369,0.053234
USC00480228,ANCHOR DAM,43.6667,-108.8333,1969.9,-0.006350,-0.154621,-0.076194,0.102126


In [8]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [9]:
min_lat,max_lat,min_long,max_long = box = (mlat, mxlat, mlong, mxlong)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 7
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    